<a href="https://github.com/theonaunheim">
    <img style="border-radius: 100%; float: right;" src="static/strawberry_thief_square.png" width=10% alt="Theo Naunheim's Github">
</a>

<br style="clear: both">
<hr>
<br>

<h1 align='center'>Series Part 1 Exercises</h1>

<br>

<div style="display: table; width: 100%">
    <div style="display: table-row; width: 100%;">
        <div style="display: table-cell; width: 50%; vertical-align: middle;">
            <h1 align="center"> $W=-\Delta PE$ </h1>
        </div>
        <div style="display: table-cell; width: 10%">
        </div>
        <div style="display: table-cell; width: 40%; vertical-align: top;">
            <blockquote>
                <p style="font-style: italic;">"I hear and I forget. I see and I remember. I do and I understand."</p>
                <br>
                <p>-Confucius</p>
            </blockquote>
        </div>
    </div>
</div>

<br>

<hr>

# Problem 1

* Import Numpy and Pandas under the `np` and `pd` aliases.


* Create a Series from the 'height_in_feet.csv' in the data folder and assign it to variable `height_feet`.


* Create a Series from the 'weight_in_pounds.csv' in the data folder and assign it to variable `weight_pounds`.


* Display your `height_feet` Series.

<br>

Hint: [`pd.read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) takes your CSV path as its first argument, e.g.: `'data/height_in_feet.csv'`, followed by `squeeze=True`. This will coerce your CSV to a Series (because you will generally use this method for DataFrame creation).

Hint: if you end your cell with a variable name, the Jupyter Notebook will display that item.

In [1]:
# Import stuff so we can use libraries.
import numpy as np
import pandas as pd

# Create our data series.
height_feet   = pd.read_csv('data/height_in_feet.csv'  , squeeze=True)
weight_pounds = pd.read_csv('data/weight_in_pounds.csv', squeeze=True)

# Display
height_feet

0      5.82
1      5.58
2      5.70
3      6.38
4      5.76
5      5.40
6      5.51
7      5.57
8      5.99
9      5.35
10     5.50
11     6.07
12     5.58
13     5.38
14     5.77
15     6.55
16     5.75
17     5.70
18     5.34
19     5.54
20     6.25
21     5.40
22     6.37
23     5.64
24     5.61
25     5.64
26     5.97
27     5.08
28     5.77
29     5.62
       ... 
970    6.00
971    5.78
972    5.67
973    5.84
974    5.50
975    5.67
976    5.16
977    5.43
978    5.50
979    5.58
980    5.44
981    6.08
982    5.76
983    5.90
984    5.55
985    5.77
986    5.65
987    5.41
988    5.47
989    5.89
990    5.93
991    5.60
992    6.01
993    6.43
994    5.64
995    5.74
996    5.17
997    5.37
998    5.43
999    5.76
Name: feet, Length: 1000, dtype: float64

# Problem 2

* Print the length of both Series.


* Convert `height_feet` to meters and save the resulting Series to `height_meters`. 


* Convert `weight_pounds` to kilograms and save the resulting Series to `weight_kilograms`.


* Display `height_meters`.

<br>

Hint: the regular Python function [`len()`](https://docs.python.org/3/library/functions.html?highlight=len#len) works on Series as well.

Hint: the formula for converting feet to meters is:

$feet * .3 = meters$

Hint: the formula for converting pounds to kilograms is:

$\frac{pounds}{2.2} = kilograms$

In [2]:
# Print length
count_height = len(height_feet)
count_weight = len(weight_pounds)
print(
    f'The height and weight Series are '
    f'{count_height} and {count_weight} in length, respectively.'
)

# Convert the values
height_meters = height_feet * .3
weight_kilograms = weight_pounds / 2.2

# Display height
# Don't worry about .head() yet. I'm just using it to cut down output from 1,000 to 5.
height_meters.head()

The height and weight Series are 1000 and 1000 in length, respectively.


0    1.746
1    1.674
2    1.710
3    1.914
4    1.728
Name: feet, dtype: float64

# Problem 3

* Presume that the first weight of your weight series corresponds with the first height of the height series, the second weight corresponds with the second height, etc. Use your `weight_kilograms` and `height_meters` Series to calculate the [BMI](https://en.wikipedia.org/wiki/Body_mass_index) for each of your samples and store it to the variable `bmis`.


* Get a Series of all the BMIs corresponding with obese persons (i.e. BMI above 30) and store it to varible `obese_bmis`.


* Get the [`len()`](https://docs.python.org/3/library/functions.html?highlight=len#len) of `obese_bmis`.


* Use the lengths of `obese_bmis` and `bmis` to get the percentage of obese people in your population. Print it.


* Display the `obese_bmis` Series.

<br>

Hint: BMI can be calculated via the following formula:

$bmi = \frac{kilograms}{meters^2}$

Hint: You can use boolean indexing to filter your list like this:

    # Lets get everyone over six feet in height
    bix = height_feet > 6
    taller_people = height_feet.loc[bix]

In [3]:
# Create BMI series
bmis = weight_kilograms / (height_meters) ** 2

# Get obese equal or greater than 30 using a boolean index
obese_bix = bmis >= 30
obese_bmis = bmis.loc[obese_bix]

# Alternatively
obese_bmis = bmis[bmis >= 30]

# Get percentage
len_all       = len(bmis)
len_obese     = len(obese_bmis)
percent_obese = len_obese / len_all 
print(f'The percentage of obese people is {percent_obese}')

# Display
obese_bmis.head()

The percentage of obese people is 0.286


2     34.049327
4     31.489543
5     33.619806
6     37.748907
13    30.698006
dtype: float64

# Problem 4

* Get the index values of `obese_bmi` and assign them to a variable called `patient_numbers`.

* Create a Series called `patient_series` by passing `patient_numbers` to the [`pd.Series()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.html) constructor.

* Display your new `patient_numbers` series.

<br>

Hint: you can access your index via its `index` attribute like this:
        
    # Get the index
    temp_index = height_feet.index
    
Hint: you can get the `index`'s values by accessing the index's `values` object. 

    # Get the index's values
    temp_index.values
    
    # Access the values in a single step
    height_feet.index.values

In [4]:
# Get the index values and assign to patient numbers
patient_numbers = obese_bmis.index.values

# Create patient_series using patient_numbers
patient_series = pd.Series(patient_numbers)

# Display
patient_series.head()

0     2
1     4
2     5
3     6
4    13
dtype: int64

# Problem 5

* Greate a [`pd.DatetimeIndex`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.html) starting at Jan 1, 2018, going to Dec 31, 2018, with a frequency of 'MS' (month start) and assign it to the name `ix`.


* Create a Series using `pd.Series()` and pass it `index=ix` and assign it to the variable `school_year`.


* Set the entire Series to zero using `.loc[]`.


* Display your Series.

<br>

Hint: creating a DatetimeIndex is a bit complicated, so I'm just going to give you the code.
        
    # Create datetime index
    ix = pd.DatetimeIndex(start='2018-01-01', end='2018-12-31', freq='MS')
    
Hint: you do not necessarily need to pass anything to an Series on creation. Here you will pass only the index.


Hint: the idiom for setting the entire Series to a value using the Series' loc attribute is `loc[:] = value`.

In [5]:
# Create DatetimeIndex for your Series
ix = pd.DatetimeIndex(start='2018-01-01', end='2018-12-31', freq='MS')

# Create a series with an index but no data
school_year = pd.Series(index=ix)

# Set data equal to zero
school_year.loc[:] = 0

# Problem 6

* Take your previous `school_year` Series and set months February through May to 85.0.


* Select May and the months before May and assign them to a variable `winter_semester`.


* Select the score for the month of March from `winter_semester` and print the value.


* Display `winter_semester`.

<br>


Hint: even though it works on a `DatetimeIndex`, `loc[]` is smart enough to use strings.

    # Select a range of dates returning a Series
    school_year.loc['2018-07-01': '2018-12-01']
    
    # Select a single value returning a scalar
    school_year.loc['2018-09-01']
    
    # Select an open ended range returning a Series
    school_year.loc['2018-07-01':]


In [6]:
# Set Feb, March, April, May to 85.0
school_year.loc['2018-02-01': '2018-05-01'] = 85.0

# Select May and previous months and assign to winter_semester
winter_semester = school_year.loc[:'2018-05-01']

# Print March score
print(winter_semester.loc['2018-03-01'])

# Display winter_semester
winter_semester

85.0


2018-01-01     0.0
2018-02-01    85.0
2018-03-01    85.0
2018-04-01    85.0
2018-05-01    85.0
Freq: MS, dtype: float64

# Problem 7


* Use `pd.read_csv()` to load a Series from 'data/spies.csv' and save it to the variable `spies`.


* Do the same with 'data/embassy_staff.csv' and save it to variable `staff`.


* Display `staff`.

<br>


Hint: these CSVs have index data stored with them which necessetates the use of a `index_col=0` argument with `pd.read_csv()`.

Hint: these CSVs do not have a header, so it will use your first row of data as a header unless you supply a `header=None` argument to `pd.read_csv` as well.

Hint: you will still need `squeeze=True`.

In [7]:
# Note: it's header=None ... not header=False.
# You aren't specifying the existence or not existence of a header.
# You are specifying what row the header is found in.

# Load
staff = pd.read_csv('data/embassy_staff.csv', squeeze=True, header=None, index_col=0)

# Sometimes it's easier to do this on multiple lines.
spies = pd.read_csv(
    'data/spies.csv', 
    squeeze=True, 
    header=None, 
    index_col=0
)

# Display
staff

0
Dave        30
Julius      51
Ethel       48
Jenna       25
Klaus       35
Aloysius    84
Carlos      40
Michael     28
Tito        32
Jermaine    30
Janet       28
Marlon      25
Jackie      22
Name: 1, dtype: int64

# Problem 8

* Check if each item of the `staff` Series is over 40 and assign the result to `over_fourty_bix`.


* Check if each item of the `staff.index` item is equal to 'Michael' and assign the result to `michael_bix`.


* Combine these two indexes using the `|` operator to get a bix that gives all elements over 40 and all elements equal to 'Michael'. Write the resulting Series to `combined_bix`.


* Use `combined_bix` as your indexer for the `.loc[]` object of `staff`. Write the result to result of `staff.loc[combined_bix]` to `target_staff`, a new variable.


* Display `target_staff`.

<br>


Hint: When doing comparisons (`>`) or equalities (`==`), you can simply use the vectorized logical operations (e.g. `staff == 'Carlos'`).

Hint: boolean indexes can be simply combined to create a new boolean index. Again, these boolean indexes are just Series of True and False values.

Hint: you can overwrite a Series in place (e.g. `staff = staff + 10`).


In [8]:
# Creats a boolean index (i.e. a series of True/Falses based on criterion)
over_fourty_bix = staff > 40

# Create a boolean index for 'Michael'
michael_bix = staff.index == 'Michael'

# Combine the two bixes, so it will give us everyone over 40 and Michael.
combined_bix = over_fourty_bix | michael_bix

# Overwrite staff with using the loc[combined bix].
target_staff = staff.loc[combined_bix]

# Display
target_staff

0
Julius      51
Ethel       48
Aloysius    84
Michael     28
Name: 1, dtype: int64

# Problem 9

* Create a union of the indexes of `spies` and `staff` called `union_ix`.


* Create an intersection of the indexes of `spies` and `staff` called `intersection_ix`.


* Display the intersecting values of the two indexes .


<br>

Hint: `spies.index` and `staff.index` are the objects you will be working with here.

Hint: these indexes have attached methods that return new indexes based on the inputs, such as `index_1.union(index_2)`.

In [9]:
# Get the index of spies
spies_ix = spies.index

# Get the index of staff
staff_ix = staff.index

# Create a union index (e.g. everyone) using the staff_ix.union() method.
union_ix = spies_ix.union(staff_ix)

# Create intersection index by doing the same.
intersection_ix = spies_ix.intersection(staff_ix)

# Show the intersection values.
intersection_ix

Index(['Julius', 'Ethel', 'Klaus'], dtype='object', name=0)

# Problem 10

* Take the intersection index and drop the value 'Klaus'. Assign the resulting index with the dropped element to `rosenberg_ix`.


* Create a Series under the variable `rosenberg_series` by taking `staff` and then using the `rosenberg_ix` as an indexer with `loc[]`.


* Display the `rosenberg_series`.

<br>

Hint: dropping a value from an index will return a new index that lacks that value.

In [10]:
# Drop 'Klaus' from the Rosenberg index and assign it to rosenberg_ix
rosenberg_ix = intersection_ix.drop('Klaus')

# Create a Series of Rosenbergs by using staff.loc and the rosenberg_ix
rosenberg_series = staff.loc[rosenberg_ix]

# Display the Rosenberg Series.
rosenberg_series

0
Julius    51
Ethel     48
Name: 1, dtype: int64